In [1]:
import pandas as pd
import numpy as np
import time
import random
from multiprocessing import Pool, cpu_count, Manager

from sklearn.model_selection import KFold
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import precision_recall_curve, auc, log_loss
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MultiLabelBinarizer, OneHotEncoder
from sklearn.random_projection import GaussianRandomProjection
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.neural_network import MLPClassifier
import scipy.sparse

from recsys_utils import recsys_load_training_df, recsys_evaluate, recsys_cv_split_single

df_data = recsys_load_training_df('../Data2/training_sample.tsv')
#df_data = recsys_load_training_df('../Data/training_sample_by_tweet.tsv')

In [2]:
def numeric_cat(x, nq=49):
    cat_type = pd.api.types.CategoricalDtype(categories=[*range(nq+1)], ordered=False)
    return (pd.qcut(x, nq).cat.codes + 1).astype(cat_type)

def categorical_cat(x):
    return x.astype('category')

def id_cat(x, buckets = 1000):
    return x.apply(lambda y : abs(hash(y)) % buckets)
    #return x.apply(lambda y : abs(hash(y)) % buckets).astype('category')

def encode_tweetFeature(x, replacement = ['Unknown']):
    return pd.Series([replacement if y is np.nan else y for y in x])

def encode_response(x):
    return x.notnull().astype('int8')

numeric_cols = ['engaged_with_user_follower_count', 'engaged_with_user_following_count', 
            'engaged_with_user_account_creation', 'engaging_user_follower_count', 
            'engaging_user_following_count', 'engaging_user_account_creation',
            'tweet_timestamp',]

categorical_cols = ['tweet_type', 'language', 'engaged_with_user_is_verified',  
                'engaging_user_is_verified', 'engaged_follows_engaging']

id_cols = ['tweet_id', 'engaged_with_user_id', 'engaging_user_id']

response_cols = ['reply_timestamp', 'retweet_timestamp',
             'retweet_with_comment_timestamp', 'like_timestamp']

tweet_feature_cols = ['text_tokens', 'hashtags', 'present_media', 'present_links', 'present_domains']


df_data[numeric_cols] = df_data[numeric_cols].apply(numeric_cat)
df_data[categorical_cols] = df_data[categorical_cols].apply(categorical_cat)
df_data[id_cols] = df_data[id_cols].apply(id_cat)
df_data[tweet_feature_cols] = df_data[tweet_feature_cols].apply(encode_tweetFeature)
df_data[response_cols] = df_data[response_cols].apply(encode_response)

In [3]:
df_data.head(5)

,text_tokens,hashtags,tweet_id,present_media,present_links,present_domains,tweet_type,language,tweet_timestamp,engaged_with_user_id,...,engaging_user_id,engaging_user_follower_count,engaging_user_following_count,engaging_user_is_verified,engaging_user_account_creation,engaged_follows_engaging,reply_timestamp,retweet_timestamp,retweet_with_comment_timestamp,like_timestamp
0,"[101, 56898, 137, 174, 63247, 10526, 131, 3197...",[Unknown],601,[Video],[Unknown],[Unknown],Retweet,76B8A9C3013AE6414A3E6012413CDC3B,43,975,...,692,22,46,False,48,False,0,0,0,0
1,"[101, 102463, 10230, 10105, 21040, 10169, 1281...",[Unknown],931,[Unknown],[BB422AA00380E45F312FD2CAA75F4960],[92D397F8E0F1E77B36B8C612C2C51E23],TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,3,290,...,762,7,3,False,9,False,0,0,0,0
2,"[101, 56898, 137, 11255, 22037, 10263, 168, 11...",[DB32BD91C2F1B37BE700F374A07FBC61],120,[Unknown],[2423BA02A75DB2189335DDC3FB6B74A1],[6D323BE93766E79BE423FAC5C28BE39B],Retweet,22C448FF81263D4BAF2A176145EE9EAD,26,313,...,276,29,37,False,22,False,0,0,0,0
3,"[101, 13073, 28757, 106, 100, 14120, 131, 120,...",[Unknown],334,[Video],[Unknown],[Unknown],TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,18,860,...,946,6,4,False,13,False,0,0,0,1
4,"[101, 3460, 1923, 6632, 2824, 30368, 2179, 188...",[Unknown],841,[Unknown],[Unknown],[Unknown],Quote,22C448FF81263D4BAF2A176145EE9EAD,24,509,...,830,29,33,False,3,False,0,0,0,0


# Baseline

In [4]:
def multi_label_transform(data):
    mlb = MultiLabelBinarizer(sparse_output=True)
    return mlb.fit_transform(data)

def transform_data(df_data):

    transformed_data_list = []
    for tweet_feature in tweet_feature_cols:
        transformed_data_list += [multi_label_transform(df_data[tweet_feature])]

    transformed_tweet_features = scipy.sparse.hstack(transformed_data_list)

    encoder = OneHotEncoder()

    transformed_numeric = encoder.fit_transform(df_data[numeric_cols])
    transformed_categorical = encoder.fit_transform(df_data[categorical_cols])
    transformed_id = encoder.fit_transform(df_data[id_cols])

    rng = np.random.RandomState(42)
    trun_svd = TruncatedSVD(n_components=16, random_state=rng)

    X_numeric_trans = trun_svd.fit_transform(transformed_numeric)
    X_categorical_trans = trun_svd.fit_transform(transformed_categorical )
    X_id_trans = trun_svd.fit_transform(transformed_id)
    X_tweet_features_trans = trun_svd.fit_transform(transformed_tweet_features)

    data_trans = np.hstack([X_numeric_trans, X_categorical_trans, X_id_trans, X_tweet_features_trans])
    
    return data_trans

def compute_prauc(pred, gt):
    prec, recall, thresh = precision_recall_curve(gt, pred)
    prauc = auc(recall, prec)
    return prauc

def calculate_ctr(gt):
    positive = len([x for x in gt if x == 1])
    ctr = positive/float(len(gt))
    return ctr

def compute_rce(pred, gt):
    cross_entropy = log_loss(gt, pred)
    data_ctr = calculate_ctr(gt)
    strawman_cross_entropy = log_loss(gt, [data_ctr for _ in range(len(gt))])
    return (1.0 - cross_entropy/strawman_cross_entropy)*100.0

In [5]:
def MLP_recommender_train_predict(df_train,df_test,split_type):
    
    df_train_transformed = transform_data(df_train)
    df_test_transformed = transform_data(df_test)
    
    rec_like = MLPClassifier(solver='adam', alpha=1e-5,hidden_layer_sizes=(8, 4),
                            batch_size= 400,max_iter = 400)
    rec_like.fit(df_train_transformed, df_train['like_timestamp'])
    like_accuracy = rec_like.score(df_test_transformed,df_test['like_timestamp'])

    rec_retweet  = MLPClassifier(solver='adam', alpha=1e-5,hidden_layer_sizes=(8, 4),
                            batch_size= 400,max_iter = 400)
    rec_retweet.fit(df_train_transformed, df_train['retweet_timestamp'])
    retweet_accuracy = rec_retweet.score(df_test_transformed,df_test['retweet_timestamp'])

    rec_reply  = MLPClassifier(solver='adam', alpha=1e-5,hidden_layer_sizes=(8, 4),
                            batch_size= 400,max_iter = 400)
    rec_reply.fit(df_train_transformed, df_train['reply_timestamp'])
    reply_accuracy = rec_reply.score(df_test_transformed,df_test['reply_timestamp'])
    
    rec_retweet_wc  = MLPClassifier(solver='adam', alpha=1e-5,hidden_layer_sizes=(8, 4),
                            batch_size= 400,max_iter = 400)
    rec_retweet_wc.fit(df_train_transformed, df_train['retweet_with_comment_timestamp'])
    retweet_wc_aacuracy = rec_retweet_wc.score(df_test_transformed,df_test['retweet_with_comment_timestamp'])
    
    pred_reply = rec_reply.predict(df_test_transformed)
    pred_retweet = rec_retweet.predict(df_test_transformed)
    pred_retweet_wc = rec_retweet_wc.predict(df_test_transformed)
    pred_like=rec_like.predict(df_test_transformed )
    
    gt_reply = df_test['reply_timestamp']
    gt_retweet = df_test['retweet_timestamp']
    gt_retweet_wc = df_test['retweet_with_comment_timestamp']
    gt_like = df_test['like_timestamp']

    
    results = {
        'split_type': split_type,
        'reply (PRAUC)': compute_prauc(pred_reply, gt_reply),
        'reply (CTR)': calculate_ctr(gt_reply),
        'reply (RCE)': compute_rce(pred_reply, gt_reply),

        'retweet (PRAUC)': compute_prauc(pred_retweet, gt_retweet),
        'retweet (CTR)': calculate_ctr(gt_retweet),
        'retweet (RCE)': compute_rce(pred_retweet, gt_retweet),

        'retweet_wc (PRAUC)': compute_prauc(pred_retweet_wc, gt_retweet_wc),
        'retweet_wc (CTR)': calculate_ctr(gt_retweet_wc),
        'retweet_wc (RCE)': compute_rce(pred_retweet_wc, gt_retweet_wc),

        'like (PRAUC)': compute_prauc(pred_like, gt_like),
        'like (CTR)': calculate_ctr(gt_like),
        'like (RCE)': compute_rce(pred_like, gt_like),
    }
    return results

#df_results = recsys_evaluate(df_data, recommender_train_predict, ['tweetid', 'time'])
#df_results = recsys_evaluate(df_data, MLP_recommender_train_predict, 'all', parallel = False)
#df_results

In [7]:
df = df_data

df_results = pd.DataFrame()

# recsys_cv_split_single(df)
user_counts = df['engaging_user_id'].value_counts()
df_filtered = df[~df['engaging_user_id'].isin(user_counts[user_counts < 2].index)]

df_train, df_test = train_test_split(df_filtered, stratify=df_filtered['engaging_user_id'], test_size=0.20, random_state=42)

results = MLP_recommender_train_predict(df_train, df_test,'single_random')

df_results = df_results.append(results,ignore_index=True)


# recsys_cv_split_tweetid
unique_tweet_ids = df['tweet_id'].unique()
unique_tweet_ids.sort()
n = len(unique_tweet_ids)

tweetId_to_tweetIDX = dict(zip(unique_tweet_ids, range(n)))
tweetIDX_to_tweetId = dict(zip(range(n), unique_tweet_ids))

cv = KFold(n_splits=10, shuffle=True)

for train_idx, dev_idx in cv.split(unique_tweet_ids):
    train_df = df.loc[df.tweet_id.isin(map(tweetIDX_to_tweetId.get, train_idx)),:]
    dev_df = df.loc[df.tweet_id.isin(map(tweetIDX_to_tweetId.get, dev_idx)),:]
    results = MLP_recommender_train_predict(train_df,dev_df,'tweetid')
    df_results = df_results.append(results,ignore_index=True)


# recsys_cv_split_userid
unique_user_ids = df['engaging_user_id'].unique()
unique_user_ids.sort()
m = len(unique_user_ids)

userId_to_userIDX = dict(zip(unique_user_ids, range(m)))
userIDX_to_userId = dict(zip(range(m), unique_user_ids))

cv = KFold(n_splits=10, shuffle=True)

for train_idx, dev_idx in cv.split(unique_user_ids):
    train_df = df.loc[df.engaging_user_id.isin(map(userIDX_to_userId.get, train_idx)),:]
    dev_df = df.loc[df.engaging_user_id.isin(map(userIDX_to_userId.get, dev_idx)),:]
    results = MLP_recommender_train_predict(train_df,dev_df,'userid')
    df_results = df_results.append(results,ignore_index=True)

# recsys_cv_split_time
cv = TimeSeriesSplit(n_splits=10)
df_sorted = df.sort_values(by=['tweet_timestamp'])
for ii, (train_idx, dev_idx) in enumerate(cv.split(df_sorted)):
    train_df = df_sorted.loc[train_idx,:]
    dev_df = df_sorted.loc[dev_idx,:]
    results = MLP_recommender_train_predict(train_df,dev_df,'time')
    df_results = df_results.append(results,ignore_index=True)

# recsys_cv_split_time_single(df):
cv = TimeSeriesSplit(n_splits=10)
df_sorted = df.sort_values(by=['tweet_timestamp'])
time_splits = list(cv.split(df_sorted))

train_idx, dev_idx = time_splits[len(time_splits) - 1]
train_df = df_sorted.loc[train_idx,:]
dev_df = df_sorted.loc[dev_idx,:]
results = MLP_recommender_train_predict(train_df,dev_df,'time_single')
df_results = df_results.append(results,ignore_index=True)

/home/olga.drewitz/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/olga.drewitz/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [8]:
df_results_mean = df_results.groupby('split_type').mean()
df_results_mean['agg'] = 'mean'

df_results_min = df_results.groupby('split_type').min()
df_results_min['agg'] = 'min'

df_results_max = df_results.groupby('split_type').max()
df_results_max['agg'] = 'max'
df_results_final = pd.concat([df_results_mean, df_results_min, df_results_max]).set_index(['agg'], append=True).sort_index()

In [9]:
df_results_final 

like (CTR)  like (PRAUC)   like (RCE)  reply (CTR)  \
split_type    agg                                                        
single_random max     0.537955      0.764892 -1635.532053     0.022692   
              mean    0.537955      0.764892 -1635.532053     0.022692   
              min     0.537955      0.764892 -1635.532053     0.022692   
time          max     0.553549      0.733739 -1900.277362     0.027356   
              mean    0.543113      0.713680 -2024.258657     0.023636   
              min     0.487895      0.675707 -2330.188296     0.020654   
time_single   max     0.547121      0.743630 -1809.734149     0.023116   
              mean    0.547121      0.743630 -1809.734149     0.023116   
              min     0.547121      0.743630 -1809.734149     0.023116   
tweetid       max     0.539170      0.726875 -1918.076516     0.025855   
              mean    0.534327      0.700194 -2025.328109     0.023748   
              min     0.524419      0.679718 -2151.551964     0.020489   
userid        max     0.548971      0.737229 -1798.917296     0.028399   
              mean    0.533903      0.704667 -1972.424588     0.023785   
              min     0.479882      0.661186 -2164.432157     0.020989   

                    reply (PRAUC)  reply (RCE)  retweet (CTR)  \
split_type    agg                                               
single_random max        0.511346  -623.429349       0.136960   
              mean       0.511346  -623.429349       0.136960   
              min        0.511346  -623.429349       0.136960   
time          max        0.513678  -609.294241       0.148817   
              mean       0.461818  -632.391853       0.139871   
              min        0.011558  -656.360837       0.121050   
time_single   max        0.511558  -626.277635       0.143072   
              mean       0.511558  -626.277635       0.143072   
              min        0.511558  -626.277635       0.143072   
tweetid       max        0.512927  -612.410622       0.141641   
              mean       0.411874  -631.458458       0.137141   
              min        0.010245  -644.012955       0.131876   
userid        max        0.514199  -612.552617       0.161075   
              mean       0.213354  -663.246353       0.137399   
              min        0.010494  -790.343556       0.115499   

                    retweet (PRAUC)  retweet (RCE)  retweet_wc (CTR)  \
split_type    agg                                                      
single_random max          0.422418   -1034.906885          0.006776   
              mean         0.422418   -1034.906885          0.006776   
              min          0.422418   -1034.906885          0.006776   
time          max          0.439978   -1053.146684          0.008754   
              mean         0.323091   -1115.111230          0.007126   
              min          0.176411   -1249.320732          0.005745   
time_single   max          0.417559   -1060.051096          0.006702   
              mean         0.417559   -1060.051096          0.006702   
              min          0.417559   -1060.051096          0.006702   
tweetid       max          0.335042   -1124.890548          0.008444   
              mean         0.285122   -1218.622336          0.007236   
              min          0.192134   -1375.513158          0.005771   
userid        max          0.343953   -1234.163997          0.008956   
              mean         0.242558   -1373.166915          0.007259   
              min          0.139069   -1564.840334          0.006210   

                    retweet_wc (PRAUC)  retweet_wc (RCE)  
split_type    agg                                         
single_random max             0.503388       -476.520795  
              mean            0.503388       -476.520795  
              min             0.503388       -476.520795  
time          max             0.504377       -461.005177  
              mean            0.503563       -480.880819  
              min

In [10]:
def MLP_recommender_train_predict(df_train,df_test,split_type):
    
    df_train_transformed = transform_data(df_train)
    df_test_transformed = transform_data(df_test)
    
    rec_like = MLPClassifier( )
    rec_like.fit(df_train_transformed, df_train['like_timestamp'])
    like_accuracy = rec_like.score(df_test_transformed,df_test['like_timestamp'])

    rec_retweet  = MLPClassifier(solver='adam', alpha=1e-5,hidden_layer_sizes=(128, 64, 32),
                            batch_size= 400,max_iter = 400)
    rec_retweet.fit(df_train_transformed, df_train['retweet_timestamp'])
    retweet_accuracy = rec_retweet.score(df_test_transformed,df_test['retweet_timestamp'])

    rec_reply  = MLPClassifier(solver='adam', alpha=1e-5,hidden_layer_sizes=(128, 64, 32),
                            batch_size= 400,max_iter = 400)
    rec_reply.fit(df_train_transformed, df_train['reply_timestamp'])
    reply_accuracy = rec_reply.score(df_test_transformed,df_test['reply_timestamp'])
    
    rec_retweet_wc  = MLPClassifier(solver='adam', alpha=1e-5,hidden_layer_sizes=(128, 64, 32),
                            batch_size= 400,max_iter = 400)
    rec_retweet_wc.fit(df_train_transformed, df_train['retweet_with_comment_timestamp'])
    retweet_wc_aacuracy = rec_retweet_wc.score(df_test_transformed,df_test['retweet_with_comment_timestamp'])
    
    pred_reply = rec_reply.predict(df_test_transformed)
    pred_retweet = rec_retweet.predict(df_test_transformed)
    pred_retweet_wc = rec_retweet_wc.predict(df_test_transformed)
    pred_like=rec_like.predict(df_test_transformed )
    
    gt_reply = df_test['reply_timestamp']
    gt_retweet = df_test['retweet_timestamp']
    gt_retweet_wc = df_test['retweet_with_comment_timestamp']
    gt_like = df_test['like_timestamp']

    
    results = {
        'split_type': split_type,
        'reply (PRAUC)': compute_prauc(pred_reply, gt_reply),
        'reply (CTR)': calculate_ctr(gt_reply),
        'reply (RCE)': compute_rce(pred_reply, gt_reply),

        'retweet (PRAUC)': compute_prauc(pred_retweet, gt_retweet),
        'retweet (CTR)': calculate_ctr(gt_retweet),
        'retweet (RCE)': compute_rce(pred_retweet, gt_retweet),

        'retweet_wc (PRAUC)': compute_prauc(pred_retweet_wc, gt_retweet_wc),
        'retweet_wc (CTR)': calculate_ctr(gt_retweet_wc),
        'retweet_wc (RCE)': compute_rce(pred_retweet_wc, gt_retweet_wc),

        'like (PRAUC)': compute_prauc(pred_like, gt_like),
        'like (CTR)': calculate_ctr(gt_like),
        'like (RCE)': compute_rce(pred_like, gt_like),
    }
    return results

#df_results = recsys_evaluate(df_data, recommender_train_predict, ['tweetid', 'time'])
#df_results = recsys_evaluate(df_data, MLP_recommender_train_predict, 'all', parallel = False)
#df_results


df_results = pd.DataFrame()

# recsys_cv_split_single(df)
user_counts = df['engaging_user_id'].value_counts()
df_filtered = df[~df['engaging_user_id'].isin(user_counts[user_counts < 2].index)]

df_train, df_test = train_test_split(df_filtered, stratify=df_filtered['engaging_user_id'], test_size=0.20, random_state=42)

results = MLP_recommender_train_predict(df_train, df_test,'single_random')

df_results = df_results.append(results,ignore_index=True)


# recsys_cv_split_tweetid
unique_tweet_ids = df['tweet_id'].unique()
unique_tweet_ids.sort()
n = len(unique_tweet_ids)

tweetId_to_tweetIDX = dict(zip(unique_tweet_ids, range(n)))
tweetIDX_to_tweetId = dict(zip(range(n), unique_tweet_ids))

cv = KFold(n_splits=10, shuffle=True)

for train_idx, dev_idx in cv.split(unique_tweet_ids):
    train_df = df.loc[df.tweet_id.isin(map(tweetIDX_to_tweetId.get, train_idx)),:]
    dev_df = df.loc[df.tweet_id.isin(map(tweetIDX_to_tweetId.get, dev_idx)),:]
    results = MLP_recommender_train_predict(train_df,dev_df,'tweetid')
    df_results = df_results.append(results,ignore_index=True)


# recsys_cv_split_userid
unique_user_ids = df['engaging_user_id'].unique()
unique_user_ids.sort()
m = len(unique_user_ids)

userId_to_userIDX = dict(zip(unique_user_ids, range(m)))
userIDX_to_userId = dict(zip(range(m), unique_user_ids))

cv = KFold(n_splits=10, shuffle=True)

for train_idx, dev_idx in cv.split(unique_user_ids):
    train_df = df.loc[df.engaging_user_id.isin(map(userIDX_to_userId.get, train_idx)),:]
    dev_df = df.loc[df.engaging_user_id.isin(map(userIDX_to_userId.get, dev_idx)),:]
    results = MLP_recommender_train_predict(train_df,dev_df,'userid')
    df_results = df_results.append(results,ignore_index=True)

# recsys_cv_split_time
cv = TimeSeriesSplit(n_splits=10)
df_sorted = df.sort_values(by=['tweet_timestamp'])
for ii, (train_idx, dev_idx) in enumerate(cv.split(df_sorted)):
    train_df = df_sorted.loc[train_idx,:]
    dev_df = df_sorted.loc[dev_idx,:]
    results = MLP_recommender_train_predict(train_df,dev_df,'time')
    df_results = df_results.append(results,ignore_index=True)

# recsys_cv_split_time_single(df):
cv = TimeSeriesSplit(n_splits=10)
df_sorted = df.sort_values(by=['tweet_timestamp'])
time_splits = list(cv.split(df_sorted))

train_idx, dev_idx = time_splits[len(time_splits) - 1]
train_df = df_sorted.loc[train_idx,:]
dev_df = df_sorted.loc[dev_idx,:]
results = MLP_recommender_train_predict(train_df,dev_df,'time_single')
df_results = df_results.append(results,ignore_index=True)
    
df_results_mean = df_results.groupby('split_type').mean()
df_results_mean['agg'] = 'mean'

df_results_min = df_results.groupby('split_type').min()
df_results_min['agg'] = 'min'

df_results_max = df_results.groupby('split_type').max()
df_results_max['agg'] = 'max'
df_results_final_v2 = pd.concat([df_results_mean, df_results_min, df_results_max]).set_index(['agg'], append=True).sort_index()

/home/olga.drewitz/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/olga.drewitz/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/olga.drewitz/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/olga.drewitz/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the op

In [11]:
df_results_final_v2

like (CTR)  like (PRAUC)   like (RCE)  reply (CTR)  \
split_type    agg                                                        
single_random max     0.537955      0.709302 -2031.850438     0.022692   
              mean    0.537955      0.709302 -2031.850438     0.022692   
              min     0.537955      0.709302 -2031.850438     0.022692   
time          max     0.553549      0.700103 -2090.296336     0.027356   
              mean    0.543113      0.677449 -2195.134818     0.023636   
              min     0.487895      0.622014 -2309.423169     0.020654   
time_single   max     0.547121      0.701849 -2087.553567     0.023116   
              mean    0.547121      0.701849 -2087.553567     0.023116   
              min     0.547121      0.701849 -2087.553567     0.023116   
tweetid       max     0.548471      0.707384 -2061.636204     0.025622   
              mean    0.534325      0.681658 -2165.748423     0.023761   
              min     0.526917      0.656492 -2318.708803     0.021170   
userid        max     0.552534      0.710131 -2004.885490     0.028980   
              mean    0.534411      0.687274 -2129.496687     0.023721   
              min     0.525489      0.668271 -2190.922785     0.016159   

                    reply (PRAUC)  reply (RCE)  retweet (CTR)  \
split_type    agg                                               
single_random max        0.047947 -1463.816735       0.136960   
              mean       0.047947 -1463.816735       0.136960   
              min        0.047947 -1463.816735       0.136960   
time          max        0.091199 -1127.780948       0.148817   
              mean       0.060591 -1348.637584       0.139871   
              min        0.034847 -1708.491658       0.121050   
time_single   max        0.044171 -1000.169257       0.143072   
              mean       0.044171 -1000.169257       0.143072   
              min        0.044171 -1000.169257       0.143072   
tweetid       max        0.086942 -1128.060960       0.141610   
              mean       0.059288 -1404.606938       0.137115   
              min        0.031603 -1656.059826       0.132129   
userid        max        0.097428 -1211.879252       0.165355   
              mean       0.059204 -1555.412901       0.136929   
              min        0.039831 -2252.165463       0.103536   

                    retweet (PRAUC)  retweet (RCE)  retweet_wc (CTR)  \
split_type    agg                                                      
single_random max          0.274039   -1577.374403          0.006776   
              mean         0.274039   -1577.374403          0.006776   
              min          0.274039   -1577.374403          0.006776   
time          max          0.292631   -1557.669091          0.008754   
              mean         0.240339   -1730.519959          0.007126   
              min          0.189689   -1991.936333          0.005745   
time_single   max          0.304838   -1679.221855          0.006702   
              mean         0.304838   -1679.221855          0.006702   
              min          0.304838   -1679.221855          0.006702   
tweetid       max          0.285757   -1708.871326          0.008451   
              mean         0.261199   -1955.607643          0.007239   
              min          0.245870   -2308.428327          0.006023   
userid        max          0.274887   -1877.347717          0.008432   
              mean         0.240976   -2006.534431          0.007235   
              min          0.183332   -2160.482897          0.005745   

                    retweet_wc (PRAUC)  retweet_wc (RCE)  
split_type    agg                                         
single_random max             0.003388       -799.168342  
              mean            0.003388       -799.168342  
              min             0.003388       -799.168342  
time          max             0.031467       -730.923267  
              mean            0.014302       -979.275227  
              min